In [1]:
import scipy.io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

import scipy.signal as signal
from myFilter import Filter as Filter
import os
plt.style.use('seaborn-whitegrid')

def calFFT(signal, window = 2048 , shift = False , inDB = False, half = True, normf=True, fs=None):
    
    from scipy.fftpack import fft, fftshift
    mag = np.abs(fft(signal, window) / (len(signal)/2.0))
    freq = np.linspace(0, 1, len(mag))

    if shift:
        mag = np.abs(fftshift(mag / abs(mag).max() ) )
        freq = np.linspace(-0.5, 0.5, len(mag))
        
    
    if inDB:
        mag = 20 * np.log10( mag )

    if normf == False:
        if fs == None:
            raise ValueError("Give me 'fs'")
        freq = np.linspace(0, fs, len(mag) )

    if half:
        mag = mag[:len(mag)//2]
        freq = freq[:len(freq)//2]

    return mag, freq

def matInfo(mat):
    print(mat.keys())
    for key in mat.keys():
        print(f"mat['{key}'] - {mat[key]}")

samplingRate = 1000 #hz

In [2]:
# Load mat
bands_name = {
    0 : '0_delta_theta',
    1 : '1_alpha',
    2 : '2_low_beta',
    3 : '3_high_beta',
    4 : '4_low_gamma',
    5 : '5_middle_gamma',
    6 : '6_high_gamma'
}
folder_name = f"EEG"

# Since I have to always load the mat file and squeez out the extra layer, let's just make a function for it
def get_signal(mat, key):
    return mat[key][0]

# This is how  to read the mat file
eeg_mat = scipy.io.loadmat('EEG/EEG_rest.mat')
# The original channel 0
eeg_0 = get_signal(eeg_mat, '0')
# Channel 0 with highpass
eeg_0_highpass = get_signal(eeg_mat, '0_highpass')
# Delta + Theta component on Channel 0
eeg_0_0_delta_theta = get_signal(eeg_mat, '0_0_delta_theta')
print(eeg_0_0_delta_theta.shape)

(300000,)


In [3]:
# Prepare data

# Let's get delta theta of all channel 
delta_theta = eeg_mat[f"0_0_delta_theta"]
for i in range(15):
    temp = eeg_mat[f"{i+1}_0_delta_theta"]
    delta_theta = np.concatenate([delta_theta,temp], axis=0)

print(delta_theta.shape)
# (16, 300000) is n_features, n_samples

bss = dict()

(16, 300000)


# PCA

In [4]:
from sklearn.decomposition import PCA

pca = PCA(n_components=16)
# X: array-like, shape (n_samples, n_features)
# Training data, where n_samples is the number of samples and n_features is the number of features.
# y: None
# Ignored variable.
pca_delta_theta = pca.fit_transform(delta_theta.T).T
# Check if the algorithm works
assert (pca_delta_theta == delta_theta).all() == False, f"You data is the same."
pca_delta_theta.shape
# (16, 300000)
bss['pca'] = pca_delta_theta

# ILRMA

In [5]:
import pyroomacoustics as pra

## Prepare one-shot STFT
L = 2048
hop = L // 2
win_a = pra.hann(L)
win_s = pra.transform.stft.compute_synthesis_window(win_a, hop)

## STFT ANALYSIS
# (n_samples, n_channels)
X = pra.transform.stft.analysis(delta_theta.T, L, hop, win=win_a)

# Run ILRMA
Y = pra.bss.ilrma(X, n_iter=100, n_components=16, proj_back=True) #callback=convergence_callback)
ilrma_delta_theta = pra.transform.stft.synthesis(Y, L, hop, win=win_s).T
print(y.shape)

(300032, 16)


In [6]:
bss['ilrma'] = ilrma_delta_theta

# CCA

In [7]:
# https://www.csee.umbc.edu/~liyiou1/li_ICASSP08.pdf
# https://ieeexplore.ieee.org/document/8078739
from sklearn.cross_decomposition import CCA

ECoG = scipy.io.loadmat('dataset/20120904S11_EEGECoG_Chibi_Oosugi-Naoya+Nagasaka-Yasuo+Hasegawa+Naomi_ECoG128-EEG16_mat/ECoG_rest.mat')
ECoG = ECoG['ECoG']
# (128, 300000)

# raise ValueError()
cca = CCA(n_components=16)
# Input:::
# X: array-like of shape (n_samples, n_features)
# Training vectors, where n_samples is the number of samples and n_features is the number of predictors.
# y: array-like of shape (n_samples, n_targets)
# Target vectors, where n_samples is the number of samples and n_targets is the number of response variables.

# Return:::
# x_scores if Y is not given, (x_scores, y_scores) otherwise.

cca_delta_theta = cca.fit_transform(delta_theta.T, ECoG.T)

# Check if the algorithm works
# assert (cca_delta_theta == delta_theta).all() == False, f"You data is the same."
# cca_delta_theta.shape
# # (16, 300000)
# bss['cca'] = cca_delta_theta



In [22]:
# https://devdocs.io/statsmodels/generated/statsmodels.multivariate.cancorr.cancorr
from statsmodels.multivariate.cancorr import CanCorr

temp_out = CanCorr(ECoG.T, delta_theta.T)
result = temp_out.corr_test()
print(result)

                              Cancorr results
   Canonical Correlation Wilks' lambda   Num DF     Den DF    F Value Pr > F
----------------------------------------------------------------------------
0                 0.2497        0.7244 2048.0000 4759678.7611 47.6708 0.0000
1                 0.2370        0.7726 1905.0000 4465902.6950 40.9544 0.0000
2                 0.1696        0.8186 1764.0000 4171495.9521 34.2665 0.0000
3                 0.1527        0.8428 1625.0000 3876479.1415 31.7533 0.0000
4                 0.1468        0.8629 1488.0000 3580875.1474 29.8824 0.0000
5                 0.1377        0.8819 1353.0000 3284709.2784 27.9955 0.0000
6                 0.1276        0.8990 1220.0000 2988009.4253 26.3073 0.0000
7                 0.1260        0.9139 1089.0000 2690806.2242 24.9223 0.0000
8                 0.1198        0.9286  960.0000 2393133.2268 23.2397 0.0000
9                 0.1122        0.9421  833.0000 2095027.0745 21.5471 0.0000
10                0.1002      

In [23]:
temp_out = CanCorr(ECoG.T, pca_delta_theta.T)
result = temp_out.corr_test()
print(result)

                              Cancorr results
   Canonical Correlation Wilks' lambda   Num DF     Den DF    F Value Pr > F
----------------------------------------------------------------------------
0                 0.2497        0.7244 2048.0000 4759678.7611 47.6708 0.0000
1                 0.2370        0.7726 1905.0000 4465902.6950 40.9544 0.0000
2                 0.1696        0.8186 1764.0000 4171495.9521 34.2665 0.0000
3                 0.1527        0.8428 1625.0000 3876479.1415 31.7533 0.0000
4                 0.1468        0.8629 1488.0000 3580875.1474 29.8824 0.0000
5                 0.1377        0.8819 1353.0000 3284709.2784 27.9955 0.0000
6                 0.1276        0.8990 1220.0000 2988009.4253 26.3073 0.0000
7                 0.1260        0.9139 1089.0000 2690806.2242 24.9223 0.0000
8                 0.1198        0.9286  960.0000 2393133.2268 23.2397 0.0000
9                 0.1122        0.9421  833.0000 2095027.0745 21.5471 0.0000
10                0.1002      

In [31]:
# print(y[:300000].shape)
temp_out = CanCorr(ECoG.T, y[:300000])
# temp_out = CanCorr(ECoG.T, ilrma_delta_theta)
result = temp_out.corr_test()
print(result)

(300000, 16)
                              Cancorr results
   Canonical Correlation Wilks' lambda   Num DF     Den DF    F Value Pr > F
----------------------------------------------------------------------------
0                 0.1518        0.8401 2048.0000 4759678.7611 25.6475 0.0000
1                 0.1301        0.8599 1905.0000 4465902.6950 23.8757 0.0000
2                 0.1282        0.8747 1764.0000 4171495.9521 22.8634 0.0000
3                 0.1227        0.8893 1625.0000 3876479.1415 21.7400 0.0000
4                 0.1152        0.9029 1488.0000 3580875.1474 20.6679 0.0000
5                 0.1111        0.9150 1353.0000 3284709.2784 19.7528 0.0000
6                 0.1057        0.9265 1220.0000 2988009.4253 18.8358 0.0000
7                 0.1021        0.9369 1089.0000 2690806.2242 17.9930 0.0000
8                 0.0978        0.9468  960.0000 2393133.2268 17.1263 0.0000
9                 0.0964        0.9560  833.0000 2095027.0745 16.2608 0.0000
10               

# Save to MAT

In [ ]:
if(os.path.exists('BSS') == False):
    os.mkdir('BSS')
    
scipy.io.savemat('BSS/result.mat',bss)